In [3]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
import pandas as pd
import pyarrow.parquet as pq
from io import BytesIO
from collections import defaultdict
import os

# Autenticación con Azure AD
credential = DefaultAzureCredential()

account_url = "https://datalakecomfama.blob.core.windows.net"
blob_service_client = BlobServiceClient(account_url, credential=credential)

# Acceder al contenedor y listar blobs
container_client = blob_service_client.get_container_client("bronze")
blobs = container_client.list_blobs()

#otal_files_count = sum(1 for blob in blobs)
#print(f"Total de archivos en el contenedor: {total_files_count}")
#for blob in blobs:
    #print(blob.name)

In [4]:
# Inicializar un diccionario para agrupar archivos Parquet por carpeta
parquet_files_by_folder = defaultdict(list)

# Filtrar y agrupar archivos Parquet
for blob in blobs:
    if blob.name.endswith('.parquet'):
        # Obtener la ruta de la carpeta contenedora
        folder_path = os.path.dirname(blob.name)
        parquet_files_by_folder[folder_path].append(blob.name)

# Recopilar variables de cada carpeta
variables_por_carpeta = {}

for folder, files in parquet_files_by_folder.items():
    # Leer solo el primer archivo Parquet de la carpeta
    first_parquet_file = files[0]
    blob_client = container_client.get_blob_client(first_parquet_file)
    
    # Descargar el archivo Parquet en memoria
    parquet_data = blob_client.download_blob().readall()
    
    # Leer el esquema del archivo Parquet
    parquet_file = pq.ParquetFile(BytesIO(parquet_data))
    schema = parquet_file.schema
    
    # Obtener la lista de variables (columnas)
    variables = schema.names
    
    # Guardar las variables en el diccionario
    variables_por_carpeta[folder] = variables

KeyboardInterrupt: 

In [ ]:
# Guardar las variables en un archivo de texto
with open('variables_por_carpeta.txt', 'w', encoding='utf-8') as f:
    for folder, variables in variables_por_carpeta.items():
        f.write(f"Variables en la carpeta '{folder}':\n")
        for var in variables:
            f.write(f" - {var}\n")
        f.write("\n")

print("Las variables por carpeta han sido guardadas en 'variables_por_carpeta.txt'")

In [5]:
# Inicializar un diccionario para agrupar archivos Parquet por carpeta
parquet_files_by_folder = defaultdict(list)

# Filtrar y agrupar archivos Parquet
for blob in blobs:
    if blob.name.endswith('.parquet'):
        # Obtener la ruta de la carpeta contenedora
        folder_path = os.path.dirname(blob.name)
        parquet_files_by_folder[folder_path].append(blob.name)


#Mpstrar contenido del primer parquet de una carpeta en específico
carpeta_especifica = "Salud_Continua/tratamiento" 
if carpeta_especifica in parquet_files_by_folder:
    first_parquet_file = parquet_files_by_folder[carpeta_especifica][0] 
    blob_client = container_client.get_blob_client(first_parquet_file)
    
    # Descargar el archivo Parquet en memoria
    parquet_data = blob_client.download_blob().readall()
    
    # Leer el archivo Parquet en un DataFrame
    parquet_buffer = BytesIO(parquet_data)
    df = pd.read_parquet(parquet_buffer)

In [6]:
 display(df)

,tratamiento_id,usuario_id,fecha_creacion,creado_por,fecha_actualizacion,modificado_por,tipo_diabetes,requiere_insulina,frecuencia_inyeccion_insulina,puntaje_findrisc_diabetes,...,estado_ultimos_quince_dias,fecha_ultimo_examen_inr,fecha_interaccion,atendido_por,canal_contacto,condicion_usuario,tipo_interaccion,linea,gestion_realizada,fecha_ultimo_control_rcv


In [ ]:
pd.set_option('display.max_rows', None) 
display(df['glicemia'])